`this notebook took from the below link and edited upon to our subject`
https://github.com/balak4/Optimizing-Evac-Routes

# "Historical" Tweets - from Twitter Search API using Tweepy Library <br>

_Authors: Amy Taylor and Veronica Giannotta_ <br> <br>
Preliminary steps before using Twitter's API:
1. Sign-up for a twitter account
2. Register a twitter developer account (requires email or phone number)
3. Create a developer app (I went with the name BlockedRoads)
4. Obtain your 'Access token' and 'access token secret' in the developer dashboard

Load Imports

In [3]:
import pandas as pd
import tweepy
import shutil
import jsonpickle
import json
import datetime
import config

from pandas.io.json import json_normalize

## Part 1: Prepare API credentials
**Step 1: Authenticate account and tokens through tweepy**
<br>Required credentials:
<br>`config.consumer_key` = 'bVbkbDedhEgR3li5ryFhEGqaY'
<br>`config.consumer_secret` = 'QECkiza2M9rGQomcSdkvIlnhKpQpEJQogYE5RvZcvfFhnQuTfp'
<br>`config.access_token` = '81792373-fdAnWAHmnnmLD0wtaM4UuTCMtPaWZnm2okuK1ekwv '
<br>`config.access_token_secret` = 'Gbw5KLPVE0jL87MgyVg1shGgbfyM3kylSuzU0F7KqAFDl'

In [4]:
config.consumer_key = 'cP0BqCOcEt7B1TsWa1W5EdK58'
config.consumer_secret = 'kbsOvwUGjBjyFt4kLSyIz1PFhqinKVt3dI91Mh4VCPsss3QIOi'
config.access_token = '81792373-cAKBuhV6SxKNFNppGzEey6NxkAqfOPycSWfxQslTI'
config.access_token_secret = '6FaTYiK7pn4ji2lMs8GnzA1VwGYGlhOekhxgPhCdu4lpa'

In [5]:
# authenticate account with tweepy
auth = tweepy.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_token_secret)
# auth.set_access_token(config.access_key, config.access_secret) # this is the original code but i edited as above


# create API object to pull data from twitter - and pass in authentication code
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Now we are free to make Twitter API calls

**Step 2: Verify API is working with your account**
- Run this cell to check if program is working. Output is your twitter name

In [6]:
user = api.me()
print (user.name)

cengiz imga


**Step 3: Use geo_search API to get the `place_id` for a particular location**
- `place_id` is a unique id created by Twitter that is assigned to different neighborhoods/cities/countries
- In the query parameter, type the name of the city or country you want the place_id for
- granularity = neighborhood (default) , city , admin or country
    - EX: when query = 'Boston', the place_id generated is the same for neighborhood or city ( but empty for country)

In [33]:
#  Determine the city, country, or location you want

l=[ 'Medford', 
    'Arlington',
    'Somerville',
    'Malden',
    'Winchester',
    'Stoneham',
    'Everett'          
              ]
l2 = []
for i in l:
    places = api.geo_search(query=i, 
                            granularity="city")
# i edited code by adding for loop in order to find if there is any other city(here Medford for me) in other states
    for j in range(3):
        place_id = places[j].id
        if places[j].full_name==i+', MA':
            print(j,'th full name is:', places[j].full_name)
            print('place_id is: ',place_id)
            l2.append(place_id)
print(l2)

1 th full name is: Medford, MA
place_id is:  c8e7273a81fab7c0
2 th full name is: Arlington, MA
place_id is:  47aff34722fae115
0 th full name is: Somerville, MA
place_id is:  ae845a49091f2727
0 th full name is: Malden, MA
place_id is:  75f5a403163f6f95
0 th full name is: Stoneham, MA
place_id is:  8c16a23215a1b4b3
1 th full name is: Everett, MA
place_id is:  fa48d6b6ccab7579
['c8e7273a81fab7c0', '47aff34722fae115', 'ae845a49091f2727', '75f5a403163f6f95', '8c16a23215a1b4b3', 'fa48d6b6ccab7579']


In [24]:
places[2]

Place(_api=<tweepy.api.API object at 0x10d6d25f8>, id='01ca054566a48dad', url='https://api.twitter.com/1.1/geo/id/01ca054566a48dad.json', place_type='city', name='Everett', full_name='Everett, PA', country_code='US', country='United States', contained_within=[Place(_api=<tweepy.api.API object at 0x10d6d25f8>, id='c21da2b2ed4e3be2', url='https://api.twitter.com/1.1/geo/id/c21da2b2ed4e3be2.json', place_type='admin', name='JOHNSTOWN-ALTOONA-ST COLGE', full_name='JOHNSTOWN-ALTOONA-ST COLGE', country_code='', country='', centroid=[-78.25877922237933, 40.675733], bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x10d6d25f8>, type='Polygon', coordinates=[[[-79.417553, 39.721578], [-79.417553, 41.629888], [-77.1438, 41.629888], [-77.1438, 39.721578], [-79.417553, 39.721578]]]), attributes={})], centroid=[-78.37116810657294, 40.031429], bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x10d6d25f8>, type='Polygon', coordinates=[[[-78.386849, 40.006168], [-78.386849, 40.05669], [-78

***List of search queries made:***
<br>NOTE: there must be no space between place and place_ID

    
|searchQuery| place ID |granularity| file | # of tweets| extended tweet? |
|---|---|---|---|---|---|
|'place:96683cc9126741d1 road closed' | USA |country| ./PoGo_USA_Tutorial.json20190114_7_1_36.json | 142 | - |
| 'place:1c69a67ad480e1b1 road closed' | Houston | city/neigh | ./historical_20190114_11_59.json| 15|- |
| 'Houston road closed' | NA | NA | ./historical_20190114_12_2.json | 9|- |
|'place:96683cc9126741d1 road closed' | USA |city| ./historical_20190115_14_57.json | 210 | yes |
|'place:1c69a67ad480e1b1 road closed' | Houston |city| ./historical_20190117_11_49.json | 13 | no |


## Part 2: Download Archived Tweets with the Search API
**Step 1: Use tweepy library to download tweets from Search API based on our query search terms, and save tweets as a json file**
 - Include a place ID in the query if necessary
 - Other search params that are optional: since='2019-01-03',until='2019-01-11'
 
useful resource: http://www.dealingdata.net/2016/07/23/PoGo-Series-Tweepy/

In [ ]:
'c8e7273a81fab7c0', '47aff34722fae115',
'ae845a49091f2727', '75f5a403163f6f95', '8c16a23215a1b4b3', 'fa48d6b6ccab7579

In [45]:
searchQuery = 'place: c8e7273a81fab7c0 OR place: aff34722fae115 OR'
'place: ae845a49091f2727 OR place: 75f5a403163f6f95 OR' 
'place: 8c16a23215a1b4b3 OR place: fa48d6b6ccab7579 OR' 
'#roadclosed OR #roadaccident OR #roadblock OR "road closed" OR'
'"road accident" OR "wreck" OR "road block" OR "road construction"'
'OR "roadway reduced" OR "one lane"'

tweetCount = 0

#Open a text file to save the tweets to
with open('../cimga/historical.json', 'w') as f:

    #Tell the Cursor method: we want to use Search API (api.search), and our query
    for tweet in tweepy.Cursor(api.search,q=searchQuery,
#                                tweet_mode='extended'    #comment out extended tweet if desired
                              ).items() :         

        #Verify the tweet has place info before writing 
        if tweet.place is not None:
            
            #Write the JSON format to the text file, and add one to the number of tweets we've collected
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
            tweetCount += 1

    #Display how many tweets we have collected
    print("Downloaded {0} tweets".format(tweetCount))

Downloaded 0 tweets


In [46]:
tweet.text

"I'm at @DICKS Sporting Goods in Medford, MA https://t.co/JzoUdqhxlU"

**Step 2: Append a timestamp to the end of the json file name**

In [47]:
now=datetime.datetime.now()
now

datetime.datetime(2019, 4, 16, 20, 53, 34, 457163)

In [48]:
def file_conversion():
    #create a timestamp
    now = datetime.datetime.now()
    month = '0'+str(now.month)
    day = str(now.day)
    hour = str(now.hour)
    minute = str(now.minute)
    now_str = str(now.year)+month+day+'_'+hour+'_'+minute
    
    
    #replace the name of our file with a new timestamped filename
    dest = '../cimga/historical_' + now_str + ".json"
    shutil.move('../cimga/historical.json', dest)
    
    with open(dest, "r") as f:
        status = f.readlines()
        jsons = []
        for ind in status:
            jsons.append(json.loads(ind))
    return jsons

# Uncomment out the file_conversion function to activate the file name change
# file_conversion()

In [49]:
file_conversion()

[]

**Step 3: Add the newly created json file to the list below for easy access later**

In [82]:
# Previously created json files can be accessed from this list

# json_df = pd.read_json("../data/AT_historical/PoGo_USA_Tutorial.json20190114_7_1_36.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_11_59.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190114_12_2.json", lines = True)
# json_df = pd.read_json("../data/AT_historical/historical_20190115_14_57.json", lines = True)
json_df = pd.read_json("../cimga/historical_20190415_21_9.json", lines = True)

# Part 3: Explore Tweets from json file
### Option A: Tweets --> json file --> dataframe

In [83]:
# list of columns currently available in our dataframe
json_df.columns

Index(['contributors', 'coordinates', 'created_at', 'entities',
       'favorite_count', 'favorited', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'metadata',
       'place', 'possibly_sensitive', 'retweet_count', 'retweeted', 'source',
       'text', 'truncated', 'user'],
      dtype='object')

**View all tweets from the `text` column** (for the first 7 tweets)

In [95]:
list = json_df.loc[:10, 'text']
for i in range(len(list)):
    print(i, list[i])

0 Why ain’t nobody tell me that this Old Town Road Remix slapped like that? 🤭
1 Road construction, roadway reduced to one lane in #Medford on Rt-16 Both EB/WB between Rt-28 and Rt-99 #traffic https://t.co/AoTr5OKtFq
2 Road closed intermittently in #Medford on Rt-16 Both EB/WB between Rt-28 and Rt-99 #traffic https://t.co/AoTr5OKtFq
3 @Tri_Merch Was it a road ride? If so,why?
4 I’ll say it. Am I being meme’d by the Old Town Road song?


Explore attributes of the json file

In [131]:
def PrintMembers(obj):
    for attribute in dir(obj):
        #We don't want to show built in methods of the class
        if not attribute.startswith('__'):
            print(attribute)
PrintMembers(tweet)

_api
_json
author
contributors
coordinates
created_at
destroy
entities
favorite
favorite_count
favorited
geo
id
id_str
in_reply_to_screen_name
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
is_quote_status
lang
metadata
parse
parse_list
place
possibly_sensitive
retweet
retweet_count
retweeted
retweets
source
source_url
text
truncated
user


**Shrink dataframe down to only the useful columns**

In [132]:
columns = ['coordinates', 'created_at', 'geo', 'place', 'text', 'user']
df = json_df.loc[:, columns]
df.head()

,coordinates,created_at,geo,place,text,user
0,None,2019-04-15 04:30:33,None,"{'attributes': {}, 'bounding_box': {'coordinat...",Why ain’t nobody tell me that this Old Town Ro...,"{'contributors_enabled': False, 'created_at': ..."
1,"{'coordinates': [-71.08259, 42.40478], 'type':...",2019-04-07 16:18:16,"{'coordinates': [42.40478, -71.08259], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...","Road construction, roadway reduced to one lane...","{'contributors_enabled': False, 'created_at': ..."
2,"{'coordinates': [-71.08259, 42.40478], 'type':...",2019-04-07 15:45:31,"{'coordinates': [42.40478, -71.08259], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Road closed intermittently in #Medford on Rt-1...,"{'contributors_enabled': False, 'created_at': ..."
3,None,2019-04-06 22:37:43,None,"{'attributes': {}, 'bounding_box': {'coordinat...","@Tri_Merch Was it a road ride? If so,why?","{'contributors_enabled': False, 'created_at': ..."
4,None,2019-04-06 14:48:36,None,"{'attributes': {}, 'bounding_box': {'coordinat...",I’ll say it. Am I being meme’d by the Old Town...,"{'contributors_enabled': False, 'created_at': ..."


### Explore some features of the df

In [133]:
# see all the nested information within place (for the first two tweets)
list = json_df.loc[:1, 'place']
for i in range(len(list)):
    print(i, list[i])
    print("--------")

0 {'attributes': {}, 'bounding_box': {'coordinates': [[[-71.149807, 42.396145], [-71.071833, 42.396145], [-71.071833, 42.453759], [-71.149807, 42.453759]]], 'type': 'Polygon'}, 'contained_within': [], 'country': 'United States', 'country_code': 'US', 'full_name': 'Medford, MA', 'id': 'c8e7273a81fab7c0', 'name': 'Medford', 'place_type': 'city', 'url': 'https://api.twitter.com/1.1/geo/id/c8e7273a81fab7c0.json'}
--------
1 {'attributes': {}, 'bounding_box': {'coordinates': [[[-71.149807, 42.396145], [-71.071833, 42.396145], [-71.071833, 42.453759], [-71.149807, 42.453759]]], 'type': 'Polygon'}, 'contained_within': [], 'country': 'United States', 'country_code': 'US', 'full_name': 'Medford, MA', 'id': 'c8e7273a81fab7c0', 'name': 'Medford', 'place_type': 'city', 'url': 'https://api.twitter.com/1.1/geo/id/c8e7273a81fab7c0.json'}
--------


> The dataframe that we have now has many attributes nested within other attributes. If we want to access the corrdinates nested within `coordinates` or the city nested within `place` we will need to unpack these variables using a different method. Use the next section to convert nested values into their own columns.

## Option B. Tweets --> json --> parsed --> nested dataframe

In [138]:
# Read in json file
json_df = pd.read_json("../cimga/historical_20190415_21_9.json", lines = True)

# convert json file to list of dictionaries
tweets_data = []
notParsed = []
tweets_file = open("../cimga/historical_20190415_21_9.json","r")
for line in tweets_file:    
    if line.strip():    
        try:
            tweet=json.loads(line)
            tweets_data.append(tweet)
        except:
            notParsed.append(line)
            continue
print(len(tweets_data))
print('Could not parse: ', len(notParsed))

5
Could not parse:  0


**Unpack the nested columns**

In [142]:
tweet_cols = ['coordinates', 'created_at', 
#                'full_text',
              'text','geo', 'id', 'place', 'user']

sample_tweets_dict = [{col:tweet[col] for col in tweet_cols } for tweet in tweets_data]

# This method looks for any instances of nested dictionaries (DOES NOT FLATTEN LISTS)
sample_tweets_df = pd.io.json.json_normalize(sample_tweets_dict)

sample_tweets_df.head(2)

KeyError: 'time_zone'

In [140]:
sample_tweets_df.columns

NameError: name 'sample_tweets_df' is not defined

In [43]:
# save the USA (or Houston) historical json file as df, COMMENT OUT
# df = sample_tweets_df
# df.to_csv("../data/AT_historical/USA_142tweets.csv", index = False)

Examine who the most common tweets are coming from

In [49]:
sample_tweets_df['user.screen_name'].value_counts().head()

TotalTrafficDFW    16
TotalTrafficABQ    13
TotalTrafficMIA    10
TotalTrafficPHX     8
AlgoTraffic         7
Name: user.screen_name, dtype: int64

So the Twitter user "TotalTraffic", specific to a certain city, is contributing the most tweets to situations with "road closed", and providing location place data for each tweet